In [1]:
# Packages
import tsplib95
import random
import matplotlib.pyplot as plt
import os
import gzip

URV                                                                            MESIIA

Neural and Evolutionary Computation (NEC)
Assignment 4: Optimization with Genetic Algorithm

Teachers: Dr. Jordi Duch, Dr. Sergio Gomez

Student: Natzaret Gálvez Rísquez

Implementing a genetic algorithm for the Traveling Salesman Problem (TSP) using the TSPLIB library 

In [2]:
# File paths
tsp_file_path = 'C:/Users/Gari/Desktop/Assignments_NEC/A4/eil101.tsp'

# Load the dataset
dataset = tsplib95.load(tsp_file_path)

Genetic algorithm  for solving the Traveling Salesman Problem (TSP) using permutation representation and various selection, mutation, and crossover techniques

In [15]:
import numpy as np

def euclidean_distance(coord1, coord2):
    return np.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

def compute_distances(node_coords):
    num_cities = len(node_coords)
    distances = np.zeros((num_cities, num_cities))
    for i in range(1, num_cities + 1):
        for j in range(1, num_cities + 1):
            distances[i - 1, j - 1] = euclidean_distance(node_coords[i], node_coords[j])
    return distances

def parse_dataset(dataset):
    node_coords = {}
    for node in dataset.node_coords:
        node_coords[node] = dataset.node_coords[node]
    return node_coords

def total_distance(solution, distances):
    n = len(solution)
    solution = np.array(solution)  # Convert solution to a NumPy array
    distance = 0
    for i in range(n):
        distance += distances[solution[i], solution[(i + 1) % n]]
    return distance

def roulette_wheel_selection(population, distances):
    fitness_values = np.array([1 / total_distance(individual, distances) for individual in population])
    probabilities = fitness_values / np.sum(fitness_values)
    selected_index = np.random.choice(len(population), p=probabilities)
    return population[selected_index]

def tournament_selection(population, distances, tournament_size=5):
    population_flat = [individual for individual in population]  # Flatten the population list
    tournament_candidates_indices = np.random.choice(len(population_flat), size=tournament_size, replace=False)
    tournament_candidates = [list(population_flat[index]) for index in tournament_candidates_indices]
    tournament_fitness = [1 / total_distance(individual, distances) for individual in tournament_candidates]
    winner_index = np.argmax(tournament_fitness)
    return tournament_candidates[winner_index]

def partially_mapped_crossover(parent1, parent2):
    n = len(parent1)
    start, end = sorted(np.random.choice(n, 2, replace=False))
    child1, child2 = [-1] * n, [-1] * n
    child1[start:end + 1] = parent1[start:end + 1]
    child2[start:end + 1] = parent2[start:end + 1]

    for i in range(n):
        if not np.any(np.equal(parent2[i], child1)):
            index = i
            while np.any(child1[index] != -1):
                index = np.where(parent1 == parent2[index])[0][0]
            child1[index] = parent2[i]
        if not np.any(np.equal(parent1[i], child2)):
            index = i
            while np.any(child2[index] != -1):
                index = np.where(parent2 == parent1[index])[0][0]
            child2[index] = parent1[i]

    return child1, child2

def order_crossover(parent1, parent2):
    n = len(parent1)
    start, end = sorted(np.random.choice(n, 2, replace=False))
    child1, child2 = [-1] * n, [-1] * n
    child1[start:end + 1] = parent1[start:end + 1]
    child2[start:end + 1] = parent2[start:end + 1]

    for i in range(n):
        if child1[i] == -1:
            idx = (end + 1) % n
            while parent2[idx] in child1:
                idx = (idx + 1) % n
            child1[i] = parent2[idx]

        if child2[i] == -1:
            idx = (end + 1) % n
            while parent1[idx] in child2:
                idx = (idx + 1) % n
            child2[i] = parent1[idx]

    return child1, child2

def inversion_mutation(solution):
    n = len(solution)
    start, end = sorted(np.random.choice(n, 2, replace=False))
    return np.concatenate((solution[:start], np.flip(solution[start:end + 1]), solution[end + 1:]))

def swap_mutation(solution):
    n = len(solution)
    idx1, idx2 = np.random.choice(n, 2, replace=False)
    solution[idx1], solution[idx2] = solution[idx2], solution[idx1]
    return solution

def genetic_algorithm(distances, population_size=100, max_generations=100):
    n = distances.shape[0]
    population = [list(np.random.permutation(range(n))) for _ in range(population_size)]  # Convert arrays to lists

    for generation in range(max_generations):
        # Selection
        selected_parents = [roulette_wheel_selection(population, distances), tournament_selection(population, distances)]

        # Crossover
        child1_pmx, child2_pmx = partially_mapped_crossover(selected_parents[0], selected_parents[1])
        child1_ox, child2_ox = order_crossover(selected_parents[0], selected_parents[1])

        # Mutation
        mutation_rate = 0.01
        if np.random.rand() < mutation_rate:
            child1_pmx = inversion_mutation(child1_pmx)
        if np.random.rand() < mutation_rate:
            child2_pmx = inversion_mutation(child2_pmx)
        if np.random.rand() < mutation_rate:
            child1_ox = swap_mutation(child1_ox)
        if np.random.rand() < mutation_rate:
            child2_ox = swap_mutation(child2_ox)

        # Replace the worst individuals with the children
        distances_population = [total_distance(individual, distances) for individual in population]
        worst1_idx = np.argmax(distances_population)
        population.pop(worst1_idx)

        distances_population = [total_distance(individual, distances) for individual in population]
        worst2_idx = np.argmax(distances_population)
        population.pop(worst2_idx)

        population.extend([child1_pmx, child2_pmx, child1_ox, child2_ox])

        # Elitism: Preserve the best solutions
        population.sort(key=lambda x: total_distance(x, distances))
        elite_count = int(0.1 * population_size)  # 10% of the population considered as elite
        elite = population[:elite_count]
        population = elite + population[elite_count:]

    best_solution = min(population, key=lambda x: total_distance(x, distances))
    return best_solution

Best tour: [88, 51, 5, 20, 96, 71, 13, 15, 43, 38, 77, 78, 54, 28, 1, 56, 16, 2, 32, 4, 7, 42, 36, 3, 62, 82, 6, 29, 95, 60, 70, 64, 22, 39, 14, 53, 31, 47, 25, 45, 17, 40, 74, 65, 69, 68, 94, 89, 100, 83, 8, 19, 48, 34, 9, 99, 97, 72, 86, 23, 37, 52, 0, 79, 46, 35, 24, 10, 67, 90, 98, 91, 58, 66, 92, 84, 85, 63, 76, 55, 30, 49, 44, 87, 50, 80, 75, 73, 57, 41, 26, 61, 27, 81, 59, 33, 11, 18, 12, 93, 21]
Total distance: 3019.269683048162


In [ ]:
if __name__ == "__main__":
    # Assuming 'dataset' is provided or loaded elsewhere
    # Parse the dataset
    node_coords = parse_dataset(dataset)
    # Compute distances
    distances = compute_distances(node_coords)
    # Use genetic algorithm to find the best solution
    solution = genetic_algorithm(distances)
    # Print the best tour and its total distance
    print("Best tour:", solution)
    print("Total distance:", total_distance(solution, distances))